In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib

from shapely.geometry import Point, shape

In [2]:
dhs_filepath = '../data/dhs_info_for_climate_extraction_26sept2018.csv'
dhs_df = pd.read_csv(dhs_filepath, sep=',')
dhs_df['outlier'] = 0

In [3]:
shp_filepath = '../data/shapefiles/natural_earth_vector/10m_cultural/sub_saharan_africa.shp'
shp_gdf = gpd.read_file(shp_filepath)

Given a row of the DHS survey dataframe, the coordinates are checked against the country's respective polygon. 

If the Point(longnum, latnum) instance is not within the country's polygon, it is then marked as an outlier in the dhs_survey dataframe.

In [4]:
# Given a row of the dhs_survey dataframe, the coordinates are checked agains the 
def check_outlier(curr_row, polygon, index, dhs_df):
    curr_point = Point(curr_row.longnum, curr_row.latnum)
    # Verifying that the point instance exists withink the polygon
    if not polygon.contains(curr_point).bool():
        curr_row.outlier = 1
        dhs_df.iloc[index] = curr_row
        #print('Updated dataframe at', index)

In [3]:
#test_limit = 15000
for i, row in dhs_df.iterrows():
    check_outlier(row, shp_gdf[shp_gdf.dhscc == row.dhscc].geometry, i, dhs_df)
    #if i == test_limit:
        #break

In [4]:
dhs_df.to_csv('../data/dhs_info_for_climate_extraction_26sept2018_outliers.csv')

In [12]:
percent_out = len(dhs_df[dhs_df.outlier == 1])/len(dhs_df)
print(percent_out, '% of the DHS_survey data has been flagged as an outlier')

# Export to csv file
dhs_df.to_csv('../data/dhs_info_for_climate_extraction_26sept2018_outliers.csv')

0.012702367895881165 % of the DHS_survey data has been flagged as an outlier


In [5]:
non_outliers = dhs_df[dhs_df.outlier == 0]
non_outliers.head()